# 데이터 불러오기

In [ ]:
import sklearn
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/Shareddrives/Multicampus/Preprocessing/"
dummy = pd.read_csv(path+"0414_dummy(최종).csv")
# pivot12_mnmx = pd.read_csv(path+'pivot12_mnmx.csv')
pivot12_under = pd.read_csv(path+"pivot12_mnmx_under.csv")
pivot12_under_ind = pd.read_csv(path+"pivot12_mnmx_under_ind.csv")

In [ ]:
dummy.drop(['new_code','AGEG1','AGEG2'],inplace=True, axis=1)

In [ ]:
u_dummy_pivot12_ind = pd.merge(dummy, pivot12_under_ind, how='inner', on='SUBJECT_ID')

# 패키지 import

In [ ]:
# 사망여부 
# pipeline
from sklearn.model_selection import train_test_split

# imputer
from sklearn.impute import SimpleImputer

# scaler
from sklearn.preprocessing import StandardScaler

# model
from sklearn.svm import SVC

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 체류기간
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.impute import SimpleImputer

# 사망여부
- mean 대체
- SVC

In [ ]:
def train_test_scaler(data, scaler=StandardScaler()):
  # X, y 분리
  X = data.drop(['SUBJECT_ID','HOSPITAL_EXPIRE_FLAG','staytime'],axis=1)
  y = data['HOSPITAL_EXPIRE_FLAG']
  col_names = X.columns
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  # scaler
  sc = scaler
  sc.fit(X_train)
  X_train = sc.transform(X_train)
  X_test = sc.transform(X_test)
  
  return X_train, X_test, y_train, y_test, col_names

In [ ]:
X_train, X_test, y_train, y_test, col_names = train_test_scaler(u_dummy_pivot12_ind)

imputer = SimpleImputer()
imputed_tr = imputer.fit_transform(X_train)
imputed_ts = imputer.fit_transform(X_test)
X_train = pd.DataFrame(imputed_tr, columns=col_names)
X_test = pd.DataFrame(imputed_ts, columns=col_names)

In [ ]:
import random
random.seed(42)
idx = random.sample(range(1,len(X_test)),10)
idx

In [ ]:
X_test = X_test.iloc[idx]

In [ ]:
estimator = SVC(cache_size = 500, random_state = 42, 
                C = 10, gamma=0.001)
estimator.fit(X_train, y_train)

In [ ]:
hospital_y_pred = estimator.predict(X_test)

# 체류기간

In [ ]:
df_adm = pd.read_csv("/content/drive/Shareddrives/Multicampus/mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz")
df_dummy = pd.read_csv("/content/drive/Shareddrives/Multicampus/Preprocessing/0414_dummy(최종).csv")

In [ ]:
def prepro(df_adm, df_dummy):
    # 입원 및 퇴원 시간 datetime 형태로 변경
    df_adm['ADMITTIME'] = pd.to_datetime(df_adm['ADMITTIME'])
    df_adm['DISCHTIME'] = pd.to_datetime(df_adm['DISCHTIME'])

    # 체류시간 계산(일단위)
    df_adm['stay'] = (df_adm['DISCHTIME'] - df_adm['ADMITTIME']).dt.total_seconds() / 86400

    # 체류시간 이상치 제거
    df_adm = df_adm[df_adm['stay'] > 0]

    # 쓰지 않는 열 삭제
    df_dummy.drop(['staytime','new_code','AGEG1','AGEG2'], axis=1, inplace=True)

    # 사용할 최종 데이터 추출
    df_clean = pd.merge(df_adm[['SUBJECT_ID','stay']], df_dummy, 
                        on='SUBJECT_ID', how='inner')
    
    return df_clean

In [ ]:
def mean_z(data, target, strategy='mean'):
    content = data.drop(['SUBJECT_ID','HOSPITAL_EXPIRE_FLAG','stay'],axis=1)

    imputer = SimpleImputer(strategy=strategy)
    imputed = imputer.fit_transform(content)
    df_imputed = pd.DataFrame(imputed, columns=content.columns)
        
    X = df_imputed
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 표준화
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)

    return X_train, X_test, y_train, y_test

In [ ]:
df_clean = prepro(df_adm, df_dummy)

In [ ]:
X_train, X_test, y_train, y_test = mean_z(df_clean, 'stay')

In [ ]:
X_test = X_test[[idx]]

In [ ]:
# 모델 생성
gbr_model = GradientBoostingRegressor(random_state=0, n_estimators=180, max_depth=3, min_samples_split=60)
gbr_model.fit(X_train, y_train)

In [ ]:
stay_y_pred = gbr_model.predict(X_test)
staytime_y_pred = stay_y_pred

# 데이터 취합

In [ ]:
df = pd.DataFrame(
    data = {
        'hospital expire' : hospital_y_pred,
        'stay time' : staytime_y_pred
    })
df = df.iloc[[0,1,4]]

In [ ]:
df.info()

In [ ]:
df['hospital expire']=np.where(df['hospital expire']==0, 'expire X', 'expire O')

In [ ]:
# 시각화 위해 임의로 지정
df['SUBJECT_ID'] = ['51001','50230','52162']

In [ ]:
raw_df = df_clean[['HOSPITAL_EXPIRE_FLAG','stay']]
raw_df.columns = ['hospital expire','stay time']
raw_df.head()

In [ ]:
raw_df['hospital expire']=np.where(raw_df['hospital expire']==0, 'expire X', 'expire O')

# 시각화

In [ ]:
sns.set_theme(style="whitegrid")

fig = plt.figure(figsize=(7,9))

plt.ylim(0,50)
ax = sns.boxenplot(x="hospital expire", y="stay time",
              color="b", data=raw_df)

sns.scatterplot(x="hospital expire", y="stay time",
              data=df, hue='SUBJECT_ID', palette=['red','blue','yellow'])

In [ ]:
fig.savefig(path+'expire_staytime_classification.png')